# MiniProject - Exploring World Indicators

In [225]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
pd.set_option('display.max_rows', 1000)


In [16]:
global country_df, indicators_df

country_df = pd.read_csv('world-development-indicators/Country.csv')
indicators_df = pd.read_csv('world-development-indicators/Indicators.csv')

## Basic exploration

### Information in Countries.csv (stored in countries_df)

In [76]:
## Basic exploration
countries = country_df['ShortName'].unique()
num_countries = len(countries)
print('Found %d countries:\n%s' % (num_countries, countries))


Found 247 countries:
['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Arab World' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahrain' 'Bangladesh' 'Barbados'
 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan' 'Bolivia'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Brunei' 'Bulgaria'
 'Burkina Faso' 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada'
 'Caribbean small states' 'Cayman Islands' 'Central African Republic'
 'Central Europe and the Baltics' 'Chad' 'Channel Islands' 'Chile' 'China'
 'Colombia' 'Comoros' 'Congo' 'Costa Rica' "Côte d'Ivoire" 'Croatia'
 'Cuba' 'Curaçao' 'Cyprus' 'Czech Republic' "Dem. People's Rep. Korea"
 'Dem. Rep. Congo' 'Denmark' 'Djibouti' 'Dominica' 'Dominican Republic'
 'East Asia & Pacific (all income levels)'
 'East Asia & Pacific (developing only)' 'Ecuador' 'Egypt' 'El Salvador'
 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Ethiopia' 'Euro area'
 'Europe & Central Asia (all inc

In [80]:
## Basic exploration
keys_c = country_df.columns
num_keys_c = len(keys)
print('Found %d keys in countries.csv:\n%s\n\n' % (num_keys_c, keys_c))


Found 31 keys in countries.csv:
Index(['CountryCode', 'ShortName', 'TableName', 'LongName', 'Alpha2Code',
       'CurrencyUnit', 'SpecialNotes', 'Region', 'IncomeGroup', 'Wb2Code',
       'NationalAccountsBaseYear', 'NationalAccountsReferenceYear',
       'SnaPriceValuation', 'LendingCategory', 'OtherGroups',
       'SystemOfNationalAccounts', 'AlternativeConversionFactor',
       'PppSurveyYear', 'BalanceOfPaymentsManualInUse',
       'ExternalDebtReportingStatus', 'SystemOfTrade',
       'GovernmentAccountingConcept', 'ImfDataDisseminationStandard',
       'LatestPopulationCensus', 'LatestHouseholdSurvey',
       'SourceOfMostRecentIncomeAndExpenditureData',
       'VitalRegistrationComplete', 'LatestAgriculturalCensus',
       'LatestIndustrialData', 'LatestTradeData', 'LatestWaterWithdrawalData'],
      dtype='object')




### Information in Indicators.csv (stored in indicators_df)

In [81]:
indicators = indicators_df['IndicatorName'].unique()
num_indicators = len(indicators)

years = indicators_df['Year'].unique()
num_years = len(years)

countries_ind = indicators_df['CountryName'].unique()
num_countries_ind = len(countries_ind)

print('Data for %d years:\n%s\n\n' %(num_years, years))
print('Data for %d countries:\n%s\n\n' %(num_countries_ind, countries_ind))

print('Found %d unique indicators across %d countries and %d years' % (num_indicators, num_countries_ind, num_years))
print("Here are the indicators:\n%s" % indicators)


Data for 56 years:
[1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973
 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987
 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001
 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]


Data for 247 countries:
['Arab World' 'Caribbean small states' 'Central Europe and the Baltics'
 'East Asia & Pacific (all income levels)'
 'East Asia & Pacific (developing only)' 'Euro area'
 'Europe & Central Asia (all income levels)'
 'Europe & Central Asia (developing only)' 'European Union'
 'Fragile and conflict affected situations'
 'Heavily indebted poor countries (HIPC)' 'High income'
 'High income: nonOECD' 'High income: OECD'
 'Latin America & Caribbean (all income levels)'
 'Latin America & Caribbean (developing only)'
 'Least developed countries: UN classification' 'Low & middle income'
 'Low income' 'Lower middle income'
 'Middle East & North Africa (all income levels)'
 

In [110]:
#Function to get the country code of a country
def get_country_code(country_name):
    country_pattern = '.*' + country_name + '.*'
    b = country_df['LongName'].str.contains(country_pattern, case = False, regex = True)
    
    if b.any():
        country_info = country_df[b]
        #return country_info['CountryCode'].values[0]
        return country_info['CountryCode'].values
    else:
        b = country_df['TableName'].str.contains(country_pattern, case = False, regex = True)
        if b.any():
            country_info = country_df[b]
            #return country_info['CountryCode'].values[0]
            return country_info['CountryCode'].values
        else:
            return np.array([])

#Test function
get_country_code('mexico')

array(['MEX'], dtype=object)

In [158]:
#Function to get the list of indicators containing a certain pattern associated with a given country
def get_indicators(indicator_pattern, country_code):
   
    all_indicators_of_given_country = \
        pd.Series(indicators_df[indicators_df['CountryCode'] == country_code]['IndicatorName'].unique())

    indicator_pattern = '.*' + indicator_pattern + '.+'
    b = all_indicators_of_given_country.str.contains(indicator_pattern, case = False, regex = True)
    
    if b.any():
        return all_indicators_of_given_country[b]
    else:
        return np.array([])

#Test function
get_indicators('poverty', 'MEX')

666             Poverty gap at $1.90 a day (2011 PPP) (%)
667             Poverty gap at $3.10 a day (2011 PPP) (%)
668     Poverty headcount ratio at $1.90 a day (2011 P...
669     Poverty headcount ratio at $3.10 a day (2011 P...
1255    Poverty headcount ratio at national poverty li...
1259    Rural poverty headcount ratio at national pove...
1260    Urban poverty headcount ratio at national pove...
dtype: object

### Let the exploration begin

In [406]:
count = 1

indicator = input('Enter a string representing the indicator you want to explore:  ')
country_name = input('Which country\'s indicators (enter a string):  ')

#Can we find a country with the given string representing a country?
country_code = get_country_code(country_name)
if len(country_code) == 0:
    print('** Did not find any country with containing the string \"%s"' % country_name)
else:
    if len(country_code) > 1:
        print('Multiple countries found:')
        for code in country_code:
            print(code)
        country_code = input('Please type the country code you want: ')
    else:
        print(country_code)
        country_code = country_code[0]

    indicators_list = get_indicators(indicator, country_code)

    if len(indicators_list) > 0:
        print('\n\nFound %d indicators containing the string \"%s\" for %s:' \
              % (len(indicators_list), indicator,country_name))

        for ind in indicators_list:
            print('\t%d\t%s' % (count,ind))
            count += 1

    else:
        print('\n\nNo indicators were found for the given country and indicator')

Enter a string representing the indicator you want to explore:  expenditure
Which country's indicators (enter a string):  india
['IND']


Found 75 indicators containing the string "expenditure" for india:
	1	Discrepancy in expenditure estimate of GDP (constant LCU)
	2	Discrepancy in expenditure estimate of GDP (current LCU)
	3	Final consumption expenditure (constant 2005 US$)
	4	Final consumption expenditure (constant LCU)
	5	Final consumption expenditure (current LCU)
	6	Final consumption expenditure (current US$)
	7	Final consumption expenditure, etc. (% of GDP)
	8	Final consumption expenditure, etc. (constant 2005 US$)
	9	Final consumption expenditure, etc. (constant LCU)
	10	Final consumption expenditure, etc. (current LCU)
	11	Final consumption expenditure, etc. (current US$)
	12	General government final consumption expenditure (% of GDP)
	13	General government final consumption expenditure (constant 2005 US$)
	14	General government final consumption expenditure (constant LCU)
	15

### Indicators
    1) Researchers in R&D (per million people)
    2) Research and development expenditure (% of GDP)
    3) Government expenditure on education as % of GDP (%)
    
  

In [416]:

valid_years = range(1990,2012)
developed_countries = ['United States', 'United Kingdom', 'France', 'Germany', 'Australia', 'Japan']
developing_countries = ['Brazil', 'India', 'China', 'South Africa', 'Malaysia']
underdeveloped_countries = ['Rawanda', 'Columbia', 'Ecuador']
indicators_gdp = ['GDP per capita (current US$)', 'Trade (% of GDP)', 'Gross savings (% of GDP)']
indicators_gdp = ['GDP per capita (current US$)',  'Gross national expenditure (% of GDP)']

def get_indicator_data(list_of_indicators, list_of_countries, list_of_years):
    return indicators_df[\
                        (indicators_df['IndicatorName'].isin(list_of_indicators)) & \
                         (indicators_df['CountryName'].isin(list_of_countries)) & \
                        (indicators_df['Year'].isin(list_of_years))][['CountryName', 'IndicatorName', 'Year', 'Value']]


df_gdp_all = get_indicator_data(indicators_gdp, developing_countries+developed_countries, valid_years)

countries_gdp = {}
for country in developing_countries + developed_countries:
    df1 = pd.DataFrame(index = valid_years, columns = indicators_gdp)
    
    for ind in indicators_gdp:
        df1[ind] = df_gdp_all[(df_gdp_all['IndicatorName'] == ind) & \
                              (df_gdp_all['CountryName'] == country) & \
                             (df_gdp_all['Year'].isin(valid_years))].sort_values(by='Year')['Value'].values
        
    countries_gdp[country] = df1

display('India', countries_gdp['India'].corr())
display('United States', countries_gdp['United States'].corr())

countries_gdp['United States']

'India'

,GDP per capita (current US$),Gross national expenditure (% of GDP)
GDP per capita (current US$),1.000000,0.946109
Gross national expenditure (% of GDP),0.946109,1.000000


'United States'

,GDP per capita (current US$),Gross national expenditure (% of GDP)
GDP per capita (current US$),1.000000,0.858233
Gross national expenditure (% of GDP),0.858233,1.000000


,GDP per capita (current US$),Gross national expenditure (% of GDP)
1990,23954.479355,101.302013
1991,24405.164815,100.463473
1992,25492.951652,100.531250
1993,26464.852512,100.947473
1994,27776.635528,101.265414
1995,28782.175020,101.171155
1996,30068.230918,101.189798
1997,31572.690230,101.184525
1998,32949.197764,101.790142
1999,34620.928899,102.656464
